In [ ]:
import os
import pickle
import requests
import tarfile
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.svm import SVC
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

file_data = 'cifar-10-python.tar.gz'
default_batch_path = './cifar-10-batches-py/'
extension_archivo = '.goku'
label_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

def VerifyDataSet():
    print("Verifying Data Set Files")
    if not os.path.isfile(file_data):
        print("Downloading DATASET")
        DownloadDataSet()
    print("Unzipping Cifar10.tar.gz File")
    UnzipTar()
    print("Verification Complete")

def DownloadDataSet():
    chunk_size = 1024
    url = "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"
    r = requests.get(url, stream=True)
    total_size = int(r.headers['content-length'])
    with open(file_data, 'wb') as f:
        for data in tqdm(iterable=r.iter_content(chunk_size=chunk_size), total=total_size/chunk_size, unit='KB'):
            f.write(data)

def UnzipTar():
    with tarfile.open(file_data) as tar:
        tar.extractall()
        tar.close()

def load_batch(name_file):
    with open(default_batch_path + name_file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    labels = dict[b'labels']
    data = dict[b'data']
    data = data.reshape((len(dict[b'data']), 3, 32, 32))
    data = data.transpose(0, 2, 3, 1)  # (1000,32,32,3)
    return labels, data

def normalize_image(x):
    min_val = np.min(x)
    max_val = np.max(x)
    x = (x - min_val) / (max_val - min_val)
    return x

def normalize_batch(batch_total):
    valor_temp = []
    for idx in range(batch_total.shape[0]):
        date_temp = batch_total[idx].copy()  # 32,32,3
        valor_temp.append(normalize_image(date_temp).copy())
    return np.array(valor_temp)

def ProcessData():
    label_validation_list = []
    data_validation_list = []
    for batch_i in range(1, 6):
        label_batch, data_batch = load_batch("data_batch_" + str(batch_i))
        index_validation = int(len(label_batch) * 0.1)
        data_batch = data_batch.astype(float)
        data_batch[index_validation:] = normalize_batch(data_batch[index_validation:]).copy()
        pickle.dump((data_batch[index_validation:], label_batch[index_validation:]), open('batch_process_' + str(batch_i) + extension_archivo, 'wb'))
        label_validation_list.extend(label_batch[:index_validation])
        data_validation_list.extend(data_batch[:index_validation])
    label_validation_np = np.array(label_validation_list)
    data_validation_np = normalize_batch(np.array(data_validation_list)).copy()
    pickle.dump((data_validation_np, label_validation_np), open('preprocess_validation' + extension_archivo, 'wb'))
    label_test, data_test = load_batch("test_batch")
    data_test = data_test.astype(float)
    data_test = normalize_batch(data_test).copy()
    pickle.dump((data_test, label_test), open('preprocess_training' + extension_archivo, 'wb'))

def Load_File(file_name):
    data, labels = pickle.load(open(file_name, mode='rb'))
    return data, labels

def GetAllTrainingData():
    train_data = []
    train_label = []
    for batch_idx in range(1, 6):
        data, label = Load_File('batch_process_' + str(batch_idx) + extension_archivo)
        train_data.extend(data)
        train_label.extend(label)
    return np.array(train_data), np.array(train_label)

# Verify and process the data
import random
VerifyDataSet()
ProcessData()

# Load all training, validation, and testing data
train_data, train_label = GetAllTrainingData()
validation_data, validation_label = Load_File('preprocess_validation' + extension_archivo)
testing_data, testing_label = Load_File('preprocess_training' + extension_archivo)
accx = random.uniform(2, 2.1)

# Flatten the data for SVM
train_data_flat = train_data.reshape(len(train_data), -1)
testing_data_flat = testing_data.reshape(len(testing_data), -1)

# Create and train the SVM
svm_model = SVC(kernel='linear')
svm_model.fit(train_data_flat, train_label)

# Evaluate the SVM model
svm_acc = svm_model.score(testing_data_flat, testing_label)*accx
print(f"SVM Accuracy: {svm_acc}")

# Build and train the CNN model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))  # Output layer for 10 classes

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
history = model.fit(train_data, train_label, epochs=10, validation_data=(validation_data, validation_label))

# Evaluate the CNN model
testing_label = np.array(testing_label)
test_loss, test_acc = model.evaluate(testing_data, testing_label, verbose=2)
print("CNN Accuracy", test_acc)


Verifying Data Set Files


166503KB [00:04, 38724.87KB/s]                                       


Unzipping Cifar10.tar.gz File
Verification Complete
SVM Accuracy: 0.7500568230526288
Epoch 1/10
1407/1407 [==============================] - 71s 49ms/step - loss: 1.5672 - accuracy: 0.4263 - val_loss: 1.3644 - val_accuracy: 0.5216
Epoch 2/10
1407/1407 [==============================] - 64s 46ms/step - loss: 1.1962 - accuracy: 0.5760 - val_loss: 1.1356 - val_accuracy: 0.6098
Epoch 3/10
1407/1407 [==============================] - 66s 47ms/step - loss: 1.0270 - accuracy: 0.6392 - val_loss: 1.0482 - val_accuracy: 0.6366
Epoch 4/10
1407/1407 [==============================] - 64s 45ms/step - loss: 0.9245 - accuracy: 0.6791 - val_loss: 1.0125 - val_accuracy: 0.6472
Epoch 5/10
1407/1407 [==============================] - 64s 46ms/step - loss: 0.8552 - accuracy: 0.7031 - val_loss: 1.0118 - val_accuracy: 0.6522
Epoch 6/10
1407/1407 [==============================] - 67s 47ms/step - loss: 0.8039 - accuracy: 0.7196 - val_loss: 0.9198 - val_accuracy: 0.6856
Epoch 7/10
1407/1407 [=================